## Setup

In [ ]:
# Imports 
import codecs 
import csv 
import torch
from torch import optim 
from src import * # This loads all the models, utilities and vars

In [ ]:
# Determine the device to use 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device: {}".format(device))

## Data Loading and Preproecessing - DailyDialog Coprus

In [ ]:
# -- Dataset specific globals
CORPUS_NAME = 'dailydialog'
CORPUS_PATH = os.path.join('data',CORPUS_NAME)
CORPUS_TEXT_FILE_PATH = os.path.join(CORPUS_PATH,"dialogues_text.txt")
CORPUS_FORMATTED_FILE_PATH = os.path.join(
    CORPUS_PATH, 'formatted_dialogues_text.txt')
DELIMITER = str(codecs.decode('\t','unicode_escape'))

print(CORPUS_TEXT_FILE_PATH)
print(CORPUS_FORMATTED_FILE_PATH)

In [ ]:
# Test of the dataset paths symlink is working
print(os.path.isfile(CORPUS_TEXT_FILE_PATH))

In [ ]:
# Load the dailydialog dataset: https://www.aclweb.org/anthology/I17-1099/
printLines(CORPUS_TEXT_FILE_PATH,n=1)

In [ ]:
# Format the data file
print('Loading conversations from {}'.format(CORPUS_TEXT_FILE_PATH))
conversations = loadLines(CORPUS_TEXT_FILE_PATH)
print("Writing new formatted file: {}...".format(CORPUS_FORMATTED_FILE_PATH))
datafile = CORPUS_FORMATTED_FILE_PATH
with open(datafile,'w',encoding ='utf-8') as f: 
    writer = csv.writer(f, delimiter = DELIMITER, lineterminator='\n')
    for pair in utils.extractSentencePairs(conversations):
        writer.writerow(pair)
print("Written formatted file: {}".format(datafile))

In [ ]:
# Printing the formatted data 
printLines(datafile, n = 1)

## General Data Processing

In [ ]:
#  -- GLOBALS
datafile = CORPUS_FORMATTED_FILE_PATH
# List of dull responses 
DULL_RESPONSES = ["I do not know what you are talking about.", "I do not know.", 
 "You do not know.", "You know what I mean.", "I know what you mean.", 
 "You know what I am saying.", "You do not know anything."]


MAX_UTTERANCE_LENGTH = 10 # Maximum length of a sentence 
MIN_TRIM_COUNT  = 0 # Words that occur less than this are trimmed 

In [ ]:
print('Creating vocabulary and pairs...')
voc, pairs = utils.loadPrepareDataset(
    CORPUS_NAME, datafile, DULL_RESPONSES, MIN_TRIM_COUNT, MAX_UTTERANCE_LENGTH)
print("Vocabulary and pairs created!")

In [ ]:
# Example for validation
small_batch_size = 5
batches = batchToTrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

## Loading Models

In [ ]:
# -- VARS. 

# Loading vars. 
MODEL_NAME = 'cb_model'
ATTENTION_METHOD = 'dot'
HIDDEN_SIZE = 500 
ENCODER_N_LAYERS = 2 
DECODER_N_LAYERS = 2 
DROPOUT = 0.1 
BATCH_SIZE = 64
checkpoint = dict()

# Training vars.
SAVE_DIR = os.path.join('data','save') 
CLIP = 50.0 
TEACHER_FORCING_RATIO = 1.0 
LEARNING_RATE = 0.0001
DECODER_LEARNING_RATIO = 5.0 
N_ITERATIONS = 10000
PRINT_EVERY = 50
SAVE_EVERY = 500
TRAIN = True
LOAD = False

print(SAVE_DIR)

In [ ]:
# Define the saved model path 
# NOTE: THe checkpoint iter should be defined manually
CHECKPOINT_ITER = 15_000
SAVED_MODEL_PATH = os.path.join(
                SAVE_DIR, MODEL_NAME, CORPUS_NAME,
                "{}-{}_{}".format(
                ENCODER_N_LAYERS, DECODER_N_LAYERS, HIDDEN_SIZE),
                '{}_checkpoint.tar'.format(CHECKPOINT_ITER))

print(SAVED_MODEL_PATH)

In [ ]:
# Initialize layers, models, and optimizers  
print("Initializing models...")
embedding = nn.Embedding(voc.num_words, HIDDEN_SIZE)
encoder = EncoderRNN(
    HIDDEN_SIZE, embedding, ENCODER_N_LAYERS, DROPOUT)
decoder = LuongAttnDecoderRNN(
    ATTENTION_METHOD, embedding, HIDDEN_SIZE, voc.num_words, 
    DECODER_N_LAYERS, DROPOUT)
# Initialize optimizers 
encoder_optimizer = optim.Adam(encoder.parameters(), lr=LEARNING_RATE)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=LEARNING_RATE * DECODER_LEARNING_RATIO)
print("Models initialized!")

In [ ]:
# Load from the saved model file if it exists 
checkpoint = {}
if LOAD:
    checkpoint = loadModel(voc, SAVED_MODEL_PATH, embedding, 
          encoder, decoder, encoder_optimizer, decoder_optimizer)
if not 'iteration' in checkpoint:
    print("No saved model found")
    checkpoint = {'iteration' : 0} 
else:
    print("Loading from saved checkpoint: {}".format(SAVED_MODEL_PATH))

In [ ]:
# Encoder / decoder should use the right device 
encoder = encoder.to(device)
decoder = decoder.to(device)

# Configure to use Cuda if available
try:
    for state in encoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    for state in decoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()
    print("Optimizers using cuda!")
except:
    print("Optimizers not using cuda!")

## Training 

In [ ]:
# Note training will override an existing model that was loaded
if TRAIN:
    # Put the encoder and decoder in train mode
    # NOTE: Without the encoder and decoder in train mode, the 
    # training is **very** slow. This is important because 
    # it tells layers such as dropout to behave differently. 
    encoder.train()
    decoder.train()
    print("Training...")
    train_iters(device, encoder, decoder, encoder_optimizer, 
                      decoder_optimizer, voc, pairs, BATCH_SIZE, 
                     N_ITERATIONS, SAVED_MODEL_PATH, checkpoint, CLIP, 
                     PRINT_EVERY, SAVE_EVERY, SAVE_DIR, MODEL_NAME, 
                     CORPUS_NAME, ENCODER_N_LAYERS, DECODER_N_LAYERS, 
                     HIDDEN_SIZE, embedding)
    print("Training completed!")

## Language generation 

In [ ]:
# Setting deropout layers for model to eval mode. 
# NOTE: Putting the encoder / decoder in eval mode tells them 
# to not use layers such as dropout which are more important 
# in training. 
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder,device)

# evaluateInput(searcher, voc, MAX_UTTERANCE_LENGTH , device)

## Reinforcement Learning

In [ ]:
# Configure the RL training parameters 
# NOTE: Most of the parameters have been previously defined 
# and will be reused here. 

# Loading vars. 
MODEL_NAME = 'rl_model_seq'
ATTENTION_METHOD = 'dot'
HIDDEN_SIZE = 500 
ENCODER_N_LAYERS = 2 
DECODER_N_LAYERS = 2 
DROPOUT = 0.1 
BATCH_SIZE = 64
checkpoint = dict()

# Training vars.
SAVE_DIR = os.path.join('data','save') 
CLIP = 50.0 
TEACHER_FORCING_RATIO = 0.5
LEARNING_RATE = 0.0001
DECODER_LEARNING_RATIO = 5.0 
N_ITERATIONS = 500
PRINT_EVERY = 1
SAVE_EVERY = 1
QUERY_EVERY = 1 
TRAIN = True
LOAD = False

In [ ]:
# At this point, we now have a trained encoder and decoder 
# that we want to train further using the reinforcement learning 
# mechanism. 

# NOTE: The forward encoder / decoder are trained.
forward_encoder = encoder
forward_decoder = decoder
forward_encoder = forward_encoder.to(device)
forward_decoder = forward_decoder.to(device)

# NOTE: The backward encoder / decoder are NOT trained. 
backward_encoder = EncoderRNN(
    HIDDEN_SIZE, embedding, ENCODER_N_LAYERS, DROPOUT)
backward_decoder = LuongAttnDecoderRNN(
    ATTENTION_METHOD, embedding, HIDDEN_SIZE,
    voc.num_words, DECODER_N_LAYERS, DROPOUT)
backward_encoder = backward_encoder.to(device)
backward_decoder = backward_decoder.to(device)

# Initializing the optimizers 
forward_encoder_optimizer = optim.Adam(forward_encoder.parameters(), lr=LEARNING_RATE)
forward_decoder_optimizer = optim.Adam(forward_decoder.parameters(), lr=LEARNING_RATE * DECODER_LEARNING_RATIO)
backward_encoder_optimizer = optim.Adam(backward_encoder.parameters(), lr=LEARNING_RATE)
backward_decoder_optimizer = optim.Adam(backward_decoder.parameters(), lr=LEARNING_RATE * DECODER_LEARNING_RATIO)

In [ ]:
# NOTE: The checkpoint iter should be defined manually
CHECKPOINT_ITER = 5000
SAVED_MODEL_PATH = os.path.join(
                SAVE_DIR, MODEL_NAME, CORPUS_NAME,
                "{}-{}_{}".format(
                ENCODER_N_LAYERS, DECODER_N_LAYERS, HIDDEN_SIZE),
                '{}_checkpoint.tar'.format(CHECKPOINT_ITER))

print(SAVED_MODEL_PATH)

In [ ]:
# Load from the saved model file if it exists 
# NOTE: Assuming we need to load the forward encoder / decoder. 
checkpoint = {}
if LOAD:
    checkpoint = loadModel(voc, SAVED_MODEL_PATH, embedding, 
            forward_encoder, forward_decoder, forward_encoder_optimizer, 
            forward_decoder_optimizer)
if not 'iteration' in checkpoint:
    print("No saved model found")
    checkpoint = {'iteration' : 0} 
else:
    print("Loading from saved checkpoint: {}".format(SAVED_MODEL_PATH))

In [ ]:
# Set the optimizer parameters to the right device. 
try:
    # If you have cuda, configure cuda to call
    for state in forward_encoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    for state in forward_decoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    for state in backward_encoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    for state in backward_decoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()
    print("Optimizers using cuda!")
except:
    print("Optimizers not using cuda!")

In [ ]:
# Initializing the human trainer 

HUMAN_METRICS = [
    'How many of the above sentences are easy to respond to?',
    'How many of the above sentences contribute new information?',
    'How many of the above sentences have good grammar?'
]

# This gets passed into rl_iters for training. 
human_trainer = HumanTrainer(HUMAN_METRICS, QUERY_EVERY, voc) 

In [ ]:
# Start the training loop 

# TODO: The arguments here are not correct. 
if TRAIN:
    # Set the encoders / decoders to training mode for the dropout 
    # layers to be used. 
    forward_encoder.train()
    forward_decoder.train()
    backward_encoder.train()
    backward_decoder.train()
    
    rl_iters(
        device, forward_encoder, forward_encoder_optimizer, 
        forward_decoder, forward_decoder_optimizer, 
        backward_encoder, backward_encoder_optimizer, 
        backward_decoder, backward_decoder_optimizer, 
        voc, pairs, BATCH_SIZE, TEACHER_FORCING_RATIO, 
        DULL_RESPONSES, N_ITERATIONS, PRINT_EVERY, 
        SAVE_EVERY, QUERY_EVERY, SAVE_DIR, checkpoint, 
        SAVED_MODEL_PATH,
        MIN_TRIM_COUNT, MAX_UTTERANCE_LENGTH, MODEL_NAME, 
        CORPUS_NAME, ENCODER_N_LAYERS, HIDDEN_SIZE, 
        DECODER_N_LAYERS, embedding, human_trainer)

## Optimized Language Generation 


In [ ]:
forward_encoder.eval()
forward_decoder.eval()

# NOTE: The final model is not the TRAINED encoder / decoder 
searcher = GreedySearchDecoder(forward_encoder, forward_decoder,device)

evaluateInput(searcher, voc, MAX_UTTERANCE_LENGTH , device)